<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />MSc. Carsten Bauer</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Vorlesung &mdash; Programmiertechniken 4</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2020</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml)

**Themen dieses Notebooks:** "Messdaten" herunterladen, Visualisieren, Fehlerfortpflanzung, Parameterbestimmung durch Fitten, LaTeX-kompatible Ausgabe

## Beschreibung des (Gedanken-)Experiments

Im Folgenden wollen wir erarbeiten, wie man experimentelle Daten, die zum Beispiel während eines Praktikumsversuchs gewonnen wurden, mit Julia verarbeiten kann. Ein im Grundstudium häufig durchgeführtes Experiment ist das [Pohlsche Rad](https://de.wikipedia.org/wiki/Pohlsches_Rad) als Beispiel für resonante und chaotische Schwingvorgänge.

Daran angelehnt betrachten wir die folgende Differentialgleichung eines **getriebenen, gedämpften Oszillators**,

$\quad m\ddot{x} + \gamma\dot{x}+kx = F\sin(\omega t).$

### Wir machen zwei Versuche:

1. Wir schalten Dämpfung und Triebkraft aus ($\gamma = 0, F = 0$) und bestimmen die Masse $m$ und Schwingungsperiode $T$ des freien Oszillators.
2. Wir treiben den gedämpften Oszillator mit der konstanten Kraft $F$ bei variierenden Frequenzen $\omega$ und messen die Amplitude $A$ der Oszillation.

Aus den Ergebnissen zu 1 können wir die Federkonstante $k$ berechnen.

Aus den Ergebnissen zu 2 können wir die Dämpfung $\gamma$ bestimmen.

### "Messdaten" herunterladen:

**CSV Datei** ("Comma-separated values") mit Daten der Massen- und **Periodenmessung** (Versuch 1): https://tinyurl.com/y3bbrklk

**Excel Datei** mit Daten der **Resonanzmessung** (Versuch 2): https://tinyurl.com/y5t3qjkb

In [ ]:
download("http://www.thp.uni-koeln.de/trebst/Lectures/CompPhys-2020/period.csv", "period.csv")
download("http://www.thp.uni-koeln.de/trebst/Lectures/CompPhys-2020/resonance.xlsx", "resonance.xlsx");

Links: **[CSVFiles.jl](https://github.com/queryverse/CSVFiles.jl), [DataFrames.jl](https://github.com/JuliaData/DataFrames.jl)** ([Dokumentation](http://juliadata.github.io/DataFrames.jl/stable/))

Die Daten zu Versuch 2 (Resonanzmessung) liegen z.B. in einer Excel Datei `daten.xlsx` vor.

Links: **[ExcelFiles.jl](https://github.com/queryverse/ExcelFiles.jl)**

In [ ]:
resonance_data = DataFrame(load("resonance.xlsx", "Resonanzmessung!A2:C85"))

In [ ]:
ω = resonance_data[:,1]
A = resonance_data[:,2]
ΔA = resonance_data[:,3];

## Visualisieren

In [ ]:
errorbar(ω, A, yerr=ΔA, marker=".")

title("Schwingungsamplitude vs. Motorfrequenz")
xlabel("Motorfrequenz ω")
ylabel("Schwingungsamplitude A")

# 3. Federkonstante $k$ bestimmen (Automatische Fehlerfortpflanzung)

In [ ]:
using Measurements
import Measurements: value, uncertainty

Links: **[Measurements.jl](https://github.com/JuliaPhysics/Measurements.jl)** ([Dokumentation](https://juliaphysics.github.io/Measurements.jl/stable/))

Wir führen nun die Werte und Fehlerinformationen unserer Daten in `Measurements` zusammen.

Links: [Statistics](https://docs.julialang.org/en/latest/stdlib/Statistics/) (Standard Library = Paket das mit Julia ausgeliefert wird (keine Installation notwendig))

Aus `mass_mean` und `T_mean` können wir nun die Federkonstante samt Fehler bestimmen.

$T = 2\pi\sqrt{\dfrac{m}{k}} \ \  \Rightarrow \ \ k = \left( \dfrac{2\pi}{T} \right)^2 m$

## 4. Bestimmen von $\gamma$ (Fitten)

Aus der exakten Lösung der Bewegungsgleichung wissen wir, dass die Amplitude die folgende Form besitzt:

$\quad A = \dfrac{F}{\sqrt{m^2 (\frac{k}{m} - \omega^2)^2 + \gamma^2 \omega^2}}$

Bis auf die Dämpfung $\gamma$ kennen wir alle Größen. Wir können diese Relation also an die Daten fitten und $\gamma$ extrahieren.

Links: [LsqFit.jl](https://github.com/JuliaNLSolvers/LsqFit.jl)

In [ ]:
F = 1.69 # konstante Kraft
m = value(mass_mean)
k = value(k_measurement)

# Fit definieren
# p = Parameter des Fits (Array)
function model(ω, p)
    γ = p[1]
    @. F / sqrt(m^2 * (k / m - ω^2)^2 + γ^2 * ω^2)
end

# 5. Darstellung der Ergebnisse im $\LaTeX$ - Look + Speichern als PDF Datei

In [ ]:
# LaTeX einschalten
rc("text", usetex=true)
rc("font", family="serif")

# Plot
errorbar(ω, A, yerr=ΔA, fmt=".", label="Messdaten")
plot(ω, model(ω, γ), label="Fit")

title("Schwingungsamplitude vs. Motorfrequenz")
xlabel(L"Motorfrequenz $\omega$") # das L steht für LaTeX String
ylabel(L"Schwingungsamplitude $A$")
legend()

#  Als PDF Datei speichern
savefig("Schwingungsamplitude.pdf")

In [ ]:
# LaTeX ausschalten
# rc("text", usetex=false)
# rc("font", family="sans-serif")

# 6. Wo findet man externe Julia Pakete?

Eine Übersicht von Paketen gibt es z.B. unter https://juliaobserver.com/ und https://pkg.julialang.org/docs/.

Viele Pakete sind in sogenannten GitHub Organisationen, z.B. JuliaMath oder JuliaQuantum, zusammengefasst. Diese sind auf https://julialang.org/community/ unter *Julia GitHub Groups* gelistet.

Hilfe und Informationen zu Paketen findet man auf deren Webseiten (auf github). Diese findet man z.B. indem man nach "**Paketname jl**" sucht.

Beispiel DataFrames: https://github.com/JuliaData/DataFrames.jl

In [ ]:
@which curve_fit(model, ω, A, [0.5])

In [114]:
@edit curve_fit(model, ω, A, [0.5])